In [1]:
import pandas as pd
import numpy as np
import torch
from torch import nn

In [2]:
class resnet (nn.Module):
    def __init__(self,channels):
        super(resnet, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(channels,channels,kernel_size=(3,3),stride=(1,1),padding=1),
            nn.ReLU(),
            nn.Conv2d(channels,channels,kernel_size=(3,3),stride=(1,1),padding=1),
            nn.ReLU()
        )
    
    def forward (self,x):
        logits = self.conv(x)
        logits = logits + x
        return logits

In [12]:
class cnn(nn.Module):
    def __init__(self,numChannels,classes):
        super(cnn, self).__init__()
        self.fwd1 = nn.Sequential(
            nn.Conv2d(in_channels=numChannels, out_channels=32,kernel_size=(3,3),stride=(1,1),padding=0),
            nn.ReLU(),
            nn.Conv2d(in_channels=32, out_channels=64,kernel_size=(3, 3),stride=(1,1),padding=0),
            nn.ReLU(),
            nn.Conv2d(in_channels=64, out_channels=64,kernel_size=(2, 2),stride=(2,2),padding=0),
            nn.ReLU())
        self.fwd2 = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=9216, out_features=128),
            nn.ReLU(),
            nn.Linear(in_features=128, out_features=classes),
            nn.ReLU())

    def forward(self, x):
        res_block = resnet(64)
        logits = self.fwd1(x)
        logits = res_block(logits)
        logits = self.fwd2(logits)
        return logits

In [4]:
df = pd.read_csv('train.csv')

In [25]:
x = torch.tensor(np.array(df.iloc[:,1:]),dtype=torch.float)

In [5]:
model = cnn(1,10)

In [6]:
loss_fn = nn.CrossEntropyLoss()

In [7]:
y_train = df["number"].iloc[0:41280]
x_train = df.drop(columns='number').iloc[0:41280]
y_test = df["number"].iloc[41280::]
x_test = df.drop(columns='number').iloc[41280::]

In [8]:
x_mean = torch.tensor(np.mean(np.array(df.drop(columns='number')),axis=0,keepdims=True).reshape([1,1,28,28]),dtype=torch.float)
x_std = torch.tensor((np.std((np.array(df.drop(columns='number')) + 1e-4),axis=0,keepdims=True)**-1).reshape([1,1,28,28]),dtype=torch.float)

In [9]:
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, momentum=0.9)

In [13]:
for j in range (0,100):
    for i in range (1,645):
        X = (torch.tensor(np.array(x_train.iloc[64*i:64*(i+1)]).reshape([64,1,28,28]),dtype=torch.float) - x_mean)*x_std
        Y = torch.tensor(np.array(y_train.iloc[64*i:64*(i+1)]),dtype=torch.int64)
        Y = torch.nn.functional.one_hot(Y, num_classes=10).type(torch.float)

        pred = model(X)
        print(pred.shape,Y.shape)
        loss = loss_fn(pred, Y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(j)

torch.Size([64, 64, 12, 12]) torch.Size([64, 10])


RuntimeError: only batches of spatial targets supported (3D tensors) but got targets of dimension: 2

In [ ]:
df1 = pd.read_csv('test.csv')

In [ ]:
img = torch.tensor(np.array(df1.iloc[7]).reshape([1,1,28,28]),dtype=torch.float)
print(model(img).argmax())

tensor(3)


In [ ]:
size = 720
print(x_test.shape)
with torch.no_grad():
            X_test = torch.tensor(np.array(x_test).reshape([720,1,28,28]),dtype=torch.float)*(10)
            y_test = torch.tensor(np.array(y_test),dtype=torch.int64)
            Y_test = torch.nn.functional.one_hot(y_test, num_classes=10).type(torch.float)
            pred = model(X_test)
            test_loss = loss_fn(pred, Y_test).item()
            correct = (pred.argmax(1) == y_test).type(torch.float).sum().item()
            
test_loss /= size
correct /= size

print(f"Accuracy: {(100*correct)}%")

(720, 784)
Accuracy: 98.33333333333333%


In [ ]:
#PATH = "cnn_model.pt"

# Save
#torch.save(model.state_dict(), PATH)